<a href="https://colab.research.google.com/github/paulaRMachado/PUC-RIO-DTSC-MVP2/blob/main/PUC_MVP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MVP Análise de Dados e Boas Práticas**

**Nome**: Paula R A Machado

**Matrícula**: 405205000163

**Dataset**: Indian Liver Patient Records

# Descrição do Problema

O conjunto de dados *Indian Liver Patient Records* contém informações laboratoriais e clínicas de pacientes do nordeste de Andhra Pradesh, na Índia, com o objetivo principal de prever a presença ou ausência de doenças hepáticas. Para isso, o dataset reúne medidas bioquímicas como níveis enzimáticos (ALT, AST), bilirrubina, proteínas totais, albumina, além de dados demográficos como idade e sexo. A proposta deste trabalho é investigar essas características e construir um modelo capaz de identificar precocemente pacientes que necessitam de avaliação médica especializada.

## Hipóteses do Problema

As hipóteses que tracei são as seguintes:

* Pacientes com níveis elevados de bilirrubina e enzimas hepáticas (AST, ALT) têm maior probabilidade de apresentar doença hepática.

* Existe relação entre o nível de albumina e a presença da doença hepática?

* Homens são mais frequentemente diagnosticados com doença hepática do que mulheres?

* Pacientes mais velhos tendem a apresentar um risco maior para o desenvolvimento de doença hepática?

* Medidas como a proporção entre albumina e globulina são bons indicadores para diferenciar pacientes saudáveis de doentes?

## Tipo de Problema

Este é um problema de **classificação supervisionada**.

Dado um conjunto de características laboratoriais e demográficas (como níveis de bilirrubina, enzimas hepáticas, proteínas, idade e sexo), o objetivo é prever se um paciente possui doença hepática ou não. O resultado desejado é classificar cada paciente em uma das duas classes: doente ou não doente.

## Seleção de Dados

O dataset *Indian Liver Patient Records* foi obtido do repositório Kaggle e contém informações laboratoriais e demográficas de pacientes com e sem doenças hepáticas. Por se tratar de um dataset real e não-curado, foi necessário carregar o arquivo CSV bruto e inspecionar a estrutura para verificar a qualidade dos dados antes da análise. Nessa etapa inicial, selecionamos todos os atributos disponíveis para uso, pois eles são potencialmente relevantes para o problema e não há necessidade de filtragem prévia de colunas.

## Atributos do Dataset

O dataset *Indian Liver Patient Records* contém 583 amostras, sendo 416 de pacientes COM doença hepática e 167 de pacientes SEM doença hepática. Há 11 atributos:

- **Age**: Idade do paciente
- **Gender**: Sexo do Paciente
- **Total Bilirubin**: Bilirrubina Total (mg/dL)
- **Direct Bilirubin**: Bilirrubina Conjugada (mg/dL)
- **Alkaline Phosphotase**: Fosfatase Alcalina (UI/L)
- **Alamine Aminotransferase**: Alanina Aminotransferase (UI/L)
- **Aspartate Aminotransferase**: Aspartato Aminotransferase (UI/L)
- **Total Protiens**: Proteínas Totais (g/dL)
- **Albumin**: Albumina (g/dL)
- **Albumin and Globulin Ratio**: Razão Albumina/Globulina (%)
- **Dataset**: Classificação (com doença hepática ou saudável)

#Bibliotecas necessárias e carga de dados

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# 🥉 Bronze – Dados Brutos carregados diretamente do repositório
url = "https://raw.githubusercontent.com/paulaRMachado/PUC-RIO-DTSC-MVP2/refs/heads/main/indian_liver_patient.csv"
bronze = pd.read_csv(url)

In [12]:
# visualização das primeiras linhas
bronze.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


#Análise de Dados

##Total e Tipo das Instâncias

In [13]:
# Informações sobre a estrutura do dataset
bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


O dataset contém 583 amostras e 11 atributos, sendo 10 numéricos e apenas um categórico (Gender). Todos os campos estão completos, exceto Albumin_and_Globulin_Ratio, que tem 4 valores ausentes. O alvo (Dataset) está codificado numericamente e o consumo de memória é baixo (~50 KB), facilitando o processamento.

COLOCAR gráfico distribuição de dataset e gender

## Estatísticas descritivas

In [8]:
# Estatísticas dos dados numéricos
bronze.describe()

,Age,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,579.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,1.286449
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.319592,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.930000,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


A maioria das variáveis apresenta grande variabilidade (desvios-padrão elevados), como Alkaline_Phosphotase, Alamine_Aminotransferase e Aspartate_Aminotransferase. Isso sugere que os pacientes têm quadros clínicos bem heterogêneos.

Há valores máximos extremamente distantes da média em várias colunas (por exemplo, Aspartate_Aminotransferase vai até 4929 contra uma média de ~110), o que indica a presença de outliers.

Algumas variáveis têm uma distribuição assimétrica, pois a mediana (50%) é bem menor que a média, como em Total_Bilirubin e Direct_Bilirubin, sugerindo que poucos casos com níveis muito altos estão puxando a média para cima.

**Análise de Dados**
Objetivo: entender a informação disponível.

Estatísticas descritivas:

Quantos atributos e instâncias existem?
Quais são os tipos de dados dos atributos?
Verifique as primeiras linhas do dataset. Algo chama a atenção?
Há valores faltantes, discrepantes ou inconsistentes?
Faça um resumo estatístico dos atributos com valor numérico (mínimo, máximo, mediana, moda, média, desvio-padrão e número de valores ausentes). O que você percebe?
Visualizações:

Verifique a distribuição de cada atributo. O que você percebe? Dica: esta etapa pode dar ideias sobre a necessidade de transformações na etapa de preparação de dados (por exemplo, converter atributos de um tipo para outro, realizar operações de discretização, normalização, padronização, etc.).
Se for um problema de classificação, verifique a distribuição de frequência das classes. O que você percebe? Dica: esta etapa pode indicar a possível necessidade futura de balanceamento de classes.
Analise os atributos individualmente ou de forma combinada, usando os gráficos mais apropriados.

Requisitos e composição da nota:
* (1,0 pt) Execução sem erros:o notebook deve poder ser executado pelo professor do início ao fim sem erros no Google Colab.
* (2,0 pts) Documentação consistente:utilize blocos de texto que expliquem textualmente cada etapa e cada decisão do seu código, contando uma história completa e compreensível, do início ao fim.
* (1,0 pt) Código limpo:seu código deve estar legível e organizado. Devem ser utilizadas as boas práticas de codificação vistas nas disciplinas Programação orientada a objetose Engenharia de software para ciência de dados, mas não é necessário que você crie classes no seu código.
* (2,0 pts) Análise de dados:após cada gráfico, você deverá escrever um parágrafo resumindo os principais achados, analisando os resultados e levantando eventuais pontos de atenção.



NÂO ESQUECER DE SEPARAR TRAIN E TEST